In [1]:
import os
from dotenv import load_dotenv
from newspaper.images import chunk_size
from nltk.corpus.reader import documents

load_dotenv()

True

In [2]:
google_api_key = os.getenv('GEMINI_API_KEY')

if google_api_key == "":
    print("key not found")
else:
    print("key found")

key found


In [3]:
import google.generativeai as genai
from llama_index.core import SimpleDirectoryReader
from llama_index.core import  VectorStoreIndex
from llama_index.core import ServiceContext
from llama_index.core import StorageContext, load_index_from_storage
# imports
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from google.genai.types import EmbedContentConfig
from llama_index.core import Settings

from IPython.display import display



e:\Project\Q-A-System-using-Google-Gemini-API-and-LlamaIndex\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
genai.configure(api_key=google_api_key)

In [5]:
for models in genai.list_models():
    if 'generateContent' in models.supported_generation_methods:
        print(models.name)

models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models

In [6]:
for model in genai.list_models():
    if 'embedContent' in model.supported_generation_methods:
        print(model.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp
models/gemini-embedding-001


In [7]:
documents = SimpleDirectoryReader('../data/')

In [8]:
doc=documents.load_data()

In [9]:
doc[0].text

'What is machine learning?\r\nMachine learning is a branch of artificial intelligence (AI) and computer science which\r\nfocuses on the use of data and algorithms to imitate the way that humans learn,\r\ngradually improving its accuracy.\r\nIBM has a rich history with machine learning. One of its own, Arthur Samuel, is credited\r\nfor coining the term, “machine learning” with his research (link resides outside ibm.com)\r\naround the game of checkers. Robert Nealey, the self-proclaimed checkers master,\r\nplayed the game on an IBM 7094 computer in 1962, and he lost to the computer.\r\nCompared to what can be done today, this feat seems trivial, but it’s considered a major\r\nmilestone in the field of artificial intelligence.\r\nOver the last couple of decades, the technological advances in storage and processing\r\npower have enabled some innovative products based on machine learning, such as\r\nNetflix’s recommendation engine and self-driving cars.\r\nMachine learning is an important c

In [10]:
embedmodel_name = "models/text-embedding-004"
embed_model = GeminiEmbedding(
    model_name=embedmodel_name, api_key=google_api_key, title="this is a document"
)

C:\Users\asend\AppData\Local\Temp\ipykernel_25584\3246188765.py:2: DeprecationWarning: Call to deprecated class GeminiEmbedding. (Should use `llama-index-embeddings-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/embeddings/google_genai/)
  embed_model = GeminiEmbedding(


In [11]:
embeddings = embed_model.get_text_embedding("Google Gemini Embeddings.")
print(embeddings[:5])
print(f"Dimension of embeddings: {len(embeddings)}")

[0.023529466, 0.016315917, -0.06835473, 0.036938, 0.054701325]
Dimension of embeddings: 768


In [12]:
from llama_index.llms.gemini import Gemini

llm = Gemini(
    model="models/gemini-2.5-pro",
)

C:\Users\asend\AppData\Local\Temp\ipykernel_25584\4002006965.py:3: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  llm = Gemini(


In [13]:

# Apply global settings
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 800
Settings.num_output = 512
Settings.chunk_overlap = 20


In [16]:
index = VectorStoreIndex.from_documents(doc, embed_model=embed_model)

In [17]:
query_engine = index.as_query_engine(llm=llm)

In [18]:
response = query_engine.query("what is ML?")

In [19]:
print(response)

Machine learning (ML) is a branch of artificial intelligence (AI) and computer science that focuses on using data and algorithms to imitate the way humans learn, gradually improving its accuracy over time. It is a significant component of the data science field.

Through the use of statistical methods, ML algorithms are trained to make classifications or predictions and to uncover key insights in data mining projects. These insights are then used to drive decision-making within applications and businesses, ideally impacting key growth metrics.

Technological advancements have enabled the creation of innovative products based on machine learning, such as Netflix's recommendation engine and self-driving cars. The term "machine learning" is credited to Arthur Samuel.


In [20]:
print(query_engine.query("tell me about sri lanka"))

I am sorry, but the provided information does not contain any details about Sri Lanka.
